# Scrape Comments & Create CSV

URL: https://forums.edmunds.com/discussion/8/general/x/car-commercials-the-good-the-bad-and-the-annoying

### 5K comment only

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

entries = []
entry = []
urlnumber = 1

while urlnumber < 101:
    url = f'https://forums.edmunds.com/discussion/8/general/x/car-commercials-the-good-the-bad-and-the-annoying/p{urlnumber}'
    try:
        r = requests.get(url, timeout = 10) # Sending a request to access the page
    except Exception as e:
        print("Error message:",e)
        break;

    data = r.text
    
    soup = BeautifulSoup(data, 'lxml') # Getting the page source into the soup
    
    for div in soup.find_all('div'):
        entry = []
        if(div.get('class') != None and div.get('class')[0] == 'Comment'): # A single post is referred to as a comment. Each comment is a block denoted in a div tag which has a class called comment.
            ps = div.find_all('p') # gets all the tags called p to a variable ps
            aas = div.find_all('a') # gets all the tags called a to a variable aas
            spans = div.find_all('span')
            times = div.find_all('time') # used to extract the time tag which gives the iDate of the post

            concat_str = ''
            for str in aas[1].contents: # prints the contents that is between the tag start and end
                if str != "<br>" or str != "<br/>": # breaks in post which we need to work around
                    concat_str = (concat_str + ' '+ str).encode("utf-8").strip() # the format extracted is a unicode - we need a uniform structure to work with the strings
            entry.append(concat_str)

            concat_str = ''
            for str in times[0].contents:
                if str != "<br>" or str != "<br/>":
                    concat_str = (concat_str + ' '+ str).encode('iso-8859-1').strip()
            entry.append(concat_str)

            for div in div.find_all('div'):
                if (div.get('class') != None and div.get('class')[0] == 'Message'): # extracting the div tag with the class attribute as message
                    blockquotes = []
                    x = div.get_text()
                    for bl in div.find_all('blockquote'):
                        blockquotes.append(bl.get_text()) # block quote is used to get the quote made by a person. get_text helps to eliminate the hyperlinks and pulls out only the data.
                        bl.decompose()
                    # Encoding the text to ascii code by replacing the non-ascii characters
                    ascii_encoding = div.get_text().replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                    # Convert the ASCII encoding to Latin1 encoding
                    latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                    # Append the encoding bytes to output list
                    entry.append(latin1_encoding)

                    for bl in blockquotes:
                        ascii_encoding = bl.replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                        latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                        entry.append(latin1_encoding)

            entries.append(entry)
            
    urlnumber += 1

columns = ['User Name', 'Comment Date', 'Full Comment']

# Convert a list of byte to list a of string     
stringlist=[[x.decode('iso-8859-1') for x in entry] for entry in entries]
# Save the list to a csv file
with open('5KComments.csv', 'w') as output:
    writer = csv.writer(output, quoting=csv.QUOTE_ALL)
    writer.writerow(columns)
    writer.writerows(stringlist)

print ("Wrote to 5KComments.csv")

### All Comments

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

entries = []
entry = []
urlnumber = 1

while urlnumber < 168:
    url = f'https://forums.edmunds.com/discussion/8/general/x/car-commercials-the-good-the-bad-and-the-annoying/p{urlnumber}'
    try:
        r = requests.get(url, timeout = 10) # Sending a request to access the page
    except Exception as e:
        print("Error message:",e)
        break;

    data = r.text
    
    soup = BeautifulSoup(data, 'lxml') # Getting the page source into the soup
    
    for div in soup.find_all('div'):
        entry = []
    
        try:
            class_attribute = div.get('class')
            if class_attribute is not None and class_attribute[0] == 'Comment':
                ps = div.find_all('p')
                aas = div.find_all('a')
                spans = div.find_all('span')
                times = div.find_all('time')

                concat_str = ''
                for str in aas[1].contents:
                    if str != "<br>" or str != "<br/>":
                        concat_str = (concat_str + ' '+ str).encode("utf-8").strip()
                entry.append(concat_str)

                concat_str = ''
                for str in times[0].contents:
                    if str != "<br>" or str != "<br/>":
                        concat_str = (concat_str + ' '+ str).encode('iso-8859-1').strip()
                entry.append(concat_str)

                for div_message in div.find_all('div'):
                    try:
                        class_attribute_message = div_message.get('class')
                        if class_attribute_message is not None and class_attribute_message[0] == 'Message':
                            blockquotes = []
                            x = div_message.get_text()
                            for bl in div_message.find_all('blockquote'):
                                blockquotes.append(bl.get_text())
                                bl.decompose()

                            ascii_encoding = div_message.get_text().replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                            latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                            entry.append(latin1_encoding)

                            for bl in blockquotes:
                                ascii_encoding = bl.replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                                latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                                entry.append(latin1_encoding)
                    except Exception as e_message:
                        continue

                entries.append(entry)
        except Exception as e_div:
            continue

            
    urlnumber += 1

columns = ['User Name', 'Comment Date', 'Full Comment']

# Convert a list of byte to list a of string     
stringlist=[[x.decode('iso-8859-1') for x in entry] for entry in entries]
# Save the list to a csv file
with open('AllComments.csv', 'w') as output:
    writer = csv.writer(output, quoting=csv.QUOTE_ALL)
    writer.writerow(columns)
    writer.writerows(stringlist)

print ("Wrote to AllComments.csv")

**Note that the comments are downloaded on Jan. 15th, 2024.**